# Lab 8: Advanced Numerical Methods and Accuracy Assessment
## Howard's Policy Improvement, EGM, and Euler Equation Errors

---

### 🎯 Lab Philosophy

In Lab 7, we implemented the fundamental algorithms (VFI, Time Iteration). Now we focus on **computational efficiency** and **accuracy assessment** — essential skills for solving larger, more complex models.

A key principle: **Speed and accuracy are not competing goals**. Better algorithms often achieve both simultaneously.

### 📚 Coverage

**Part 1: Accelerating VFI**
-   Howard's Policy Improvement (Policy Iteration within VFI)
-   Exploiting monotonicity with binary search
-   Measuring speedups systematically

**Part 2: Endogenous Grid Method (EGM)**
-   The key insight: inverting the problem
-   Eliminating the inner optimization loop
-   Implementation for the RBC model

**Part 3: Accuracy Assessment**
-   Computing Euler equation errors
-   Visualizing error distributions
-   The $\log_{10}$ error interpretation

**Part 4: Sensitivity Analysis**
-   Grid density effects
-   Tolerance choice
-   Interpolation method comparison

**Part 5: Impulse Response Analysis**
-   Generating IRFs to productivity shocks
-   Verifying economic plausibility

---

### 🔧 The Research Architect Workflow

Today's emphasis: **Validation through multiple lenses**
1. Does the algorithm converge faster? (Speedup measurement)
2. Is the solution accurate? (Euler equation errors)
3. Is it economically sensible? (Impulse responses)
4. Is it robust? (Sensitivity analysis)

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.optimize import brentq, minimize_scalar
import time
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

print("Libraries loaded successfully!")

Libraries loaded successfully!


/Users/zfeng/Library/CloudStorage/OneDrive-UniversityofNebraska/GitHub/AI_ML_Macro/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# =============================================================================
# RBC Model Class (from Lab 7, with minor enhancements)
# =============================================================================

class RBCModel:
    """
    Real Business Cycle Model - enhanced version with utility derivatives.
    """
    
    def __init__(self, 
                 alpha=0.33,
                 beta=0.99,
                 delta=0.025,
                 sigma_crra=1.0,
                 rho=0.95,
                 sigma_eps=0.007,
                 n_z=7,
                 n_k=100):
        
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.sigma_crra = sigma_crra
        self.rho = rho
        self.sigma_eps = sigma_eps
        self.n_z = n_z
        self.n_k = n_k
        
        self._compute_steady_state()
        self._discretize_productivity()
        self._construct_capital_grid()
        
    def _compute_steady_state(self):
        r_ss = 1/self.beta - (1 - self.delta)
        self.k_ss = (self.alpha / r_ss) ** (1 / (1 - self.alpha))
        self.y_ss = self.k_ss ** self.alpha
        self.c_ss = self.y_ss - self.delta * self.k_ss
        self.i_ss = self.delta * self.k_ss
        
    def _discretize_productivity(self):
        from scipy.special import erfc
        
        sigma_z = self.sigma_eps / np.sqrt(1 - self.rho**2)
        z_max = 3 * sigma_z
        z_min = -z_max
        
        log_z_grid = np.linspace(z_min, z_max, self.n_z)
        step = log_z_grid[1] - log_z_grid[0]
        
        self.z_grid = np.exp(log_z_grid)
        self.log_z_grid = log_z_grid
        
        self.Pi = np.zeros((self.n_z, self.n_z))
        
        def std_normal_cdf(x):
            return 0.5 * erfc(-x / np.sqrt(2))
        
        for i in range(self.n_z):
            for j in range(self.n_z):
                if j == 0:
                    self.Pi[i, j] = std_normal_cdf(
                        (log_z_grid[0] + step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    )
                elif j == self.n_z - 1:
                    self.Pi[i, j] = 1 - std_normal_cdf(
                        (log_z_grid[-1] - step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    )
                else:
                    self.Pi[i, j] = std_normal_cdf(
                        (log_z_grid[j] + step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    ) - std_normal_cdf(
                        (log_z_grid[j] - step/2 - self.rho * log_z_grid[i]) / self.sigma_eps
                    )
        
    def _construct_capital_grid(self, grid_type='clustered'):
        k_min = 0.5 * self.k_ss
        k_max = 1.5 * self.k_ss
        
        if grid_type == 'uniform':
            self.k_grid = np.linspace(k_min, k_max, self.n_k)
        elif grid_type == 'clustered':
            x = np.linspace(0, 1, self.n_k)
            stretch = 2.0
            x_transformed = np.sinh(stretch * (x - 0.5)) / np.sinh(stretch * 0.5)
            x_transformed = (x_transformed + 1) / 2
            self.k_grid = k_min + (k_max - k_min) * x_transformed
    
    def utility(self, c):
        c = np.maximum(c, 1e-10)
        if self.sigma_crra == 1.0:
            return np.log(c)
        else:
            return (c ** (1 - self.sigma_crra) - 1) / (1 - self.sigma_crra)
    
    def marginal_utility(self, c):
        return np.maximum(c, 1e-10) ** (-self.sigma_crra)
    
    def inv_marginal_utility(self, mu):
        return np.maximum(mu, 1e-10) ** (-1 / self.sigma_crra)
    
    def production(self, k, z):
        return z * k ** self.alpha
    
    def marginal_product_k(self, k, z):
        return self.alpha * z * np.maximum(k, 1e-10) ** (self.alpha - 1)
    
    def gross_return(self, k, z):
        """Gross return on capital: R = MPK + 1 - delta"""
        return self.marginal_product_k(k, z) + 1 - self.delta

# Initialize the model
model = RBCModel(n_k=100, n_z=7)
print(f"Model initialized: k_ss = {model.k_ss:.4f}, c_ss = {model.c_ss:.4f}")

Model initialized: k_ss = 28.3484, c_ss = 2.3066


# Part 1: Accelerating VFI

## 1.1 Howard's Policy Improvement

**The Key Insight**: After finding the optimal policy at each iteration, we can "exploit" this policy for multiple periods before re-optimizing. This converts the expensive Bellman maximization into cheaper policy evaluations.

**Algorithm:**
```
1. Start with V_0, find optimal policy g_0
2. For each VFI iteration:
   a. Compute optimal policy g from current V
   b. For m = 1 to M (Howard steps):
      - Update V by evaluating (not optimizing!) under policy g:
        V(k,z) = u(c(k,z)) + β E[V(k',z') | z]
        where k' = g(k,z)
   c. Check convergence
```

**Why It Works**: The policy converges faster than the value function. Howard steps "catch up" the value function to the nearly-converged policy at low cost.

**Typical Speedup**: 10-100x depending on the model and number of Howard steps.

In [3]:
# =============================================================================
# 1.2 VFI with Howard's Policy Improvement
# =============================================================================

class VFIHoward:
    """
    Value Function Iteration with Howard's Policy Improvement.
    """
    
    def __init__(self, model, howard_steps=20):
        self.model = model
        self.howard_steps = howard_steps
        
        # Initialize
        self.V = np.zeros((model.n_k, model.n_z))
        self.policy_k_idx = np.zeros((model.n_k, model.n_z), dtype=int)
        self.policy_c = np.zeros((model.n_k, model.n_z))
        
        # Precompute feasible consumption for all (k, z, k') combinations
        self._precompute_consumption()
        
        self.convergence_history = []
        self.timing_info = {}
        
    def _precompute_consumption(self):
        """Precompute consumption matrix for efficiency."""
        self.consumption_matrix = np.zeros((self.model.n_k, self.model.n_z, self.model.n_k))
        self.feasible = np.zeros((self.model.n_k, self.model.n_z, self.model.n_k), dtype=bool)
        
        for i_k, k in enumerate(self.model.k_grid):
            for i_z, z in enumerate(self.model.z_grid):
                resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                for i_kp, k_prime in enumerate(self.model.k_grid):
                    c = resources - k_prime
                    self.consumption_matrix[i_k, i_z, i_kp] = c
                    self.feasible[i_k, i_z, i_kp] = c > 0
    
    def _howard_step(self):
        """
        Policy evaluation step: update V given fixed policy.
        """
        V_new = np.zeros_like(self.V)
        
        for i_k in range(self.model.n_k):
            for i_z in range(self.model.n_z):
                i_kp = self.policy_k_idx[i_k, i_z]
                c = self.consumption_matrix[i_k, i_z, i_kp]
                
                # Compute expected continuation value
                EV = np.dot(self.model.Pi[i_z, :], self.V[i_kp, :])
                
                V_new[i_k, i_z] = self.model.utility(c) + self.model.beta * EV
        
        self.V = V_new
    
    def _bellman_step(self):
        """
        Full Bellman optimization step.
        """
        V_new = np.zeros_like(self.V)
        
        for i_k in range(self.model.n_k):
            for i_z in range(self.model.n_z):
                best_val = -np.inf
                best_i_kp = 0
                
                for i_kp in range(self.model.n_k):
                    if not self.feasible[i_k, i_z, i_kp]:
                        continue
                    
                    c = self.consumption_matrix[i_k, i_z, i_kp]
                    EV = np.dot(self.model.Pi[i_z, :], self.V[i_kp, :])
                    val = self.model.utility(c) + self.model.beta * EV
                    
                    if val > best_val:
                        best_val = val
                        best_i_kp = i_kp
                
                V_new[i_k, i_z] = best_val
                self.policy_k_idx[i_k, i_z] = best_i_kp
                self.policy_c[i_k, i_z] = self.consumption_matrix[i_k, i_z, best_i_kp]
        
        self.V = V_new
    
    def solve(self, tol=1e-6, max_iter=500, verbose=True):
        """
        Solve using VFI with Howard's policy improvement.
        """
        print(f"\n=== VFI with Howard's Policy Improvement (M={self.howard_steps}) ===")
        start_time = time.time()
        
        n_bellman = 0
        n_howard = 0
        
        for iteration in range(max_iter):
            V_old = self.V.copy()
            
            # Full Bellman step (optimization)
            self._bellman_step()
            n_bellman += 1
            
            # Howard steps (policy evaluation only)
            for _ in range(self.howard_steps):
                self._howard_step()
                n_howard += 1
            
            # Check convergence
            diff = np.max(np.abs(self.V - V_old))
            self.convergence_history.append(diff)
            
            if verbose and iteration % 10 == 0:
                print(f"Iteration {iteration:4d}: ||V - V_old|| = {diff:.2e}")
            
            if diff < tol:
                elapsed = time.time() - start_time
                self.timing_info = {
                    'elapsed': elapsed,
                    'n_bellman': n_bellman,
                    'n_howard': n_howard,
                    'iterations': iteration + 1
                }
                print(f"\nConverged in {iteration+1} iterations ({elapsed:.2f} seconds)")
                print(f"Bellman steps: {n_bellman}, Howard steps: {n_howard}")
                return True
        
        return False
    
    def get_policy_k(self):
        """Return capital policy on the grid."""
        policy_k = np.zeros((self.model.n_k, self.model.n_z))
        for i_k in range(self.model.n_k):
            for i_z in range(self.model.n_z):
                policy_k[i_k, i_z] = self.model.k_grid[self.policy_k_idx[i_k, i_z]]
        return policy_k

In [4]:
# Compare VFI with and without Howard's improvement
print("="*60)
print("COMPARISON: Standard VFI vs Howard's Policy Improvement")
print("="*60)

# Standard VFI (Howard steps = 0)
vfi_standard = VFIHoward(model, howard_steps=0)
vfi_standard.solve(tol=1e-6, max_iter=1000, verbose=False)
time_standard = vfi_standard.timing_info['elapsed']

# VFI with Howard (M=20)
vfi_howard = VFIHoward(model, howard_steps=20)
vfi_howard.solve(tol=1e-6, max_iter=1000, verbose=False)
time_howard = vfi_howard.timing_info['elapsed']

print(f"\n{'Method':<30} {'Time (s)':>12} {'Speedup':>12}")
print("-"*60)
print(f"{'Standard VFI':<30} {time_standard:>12.2f} {'1.0x':>12}")
print(f"{'VFI + Howard (M=20)':<30} {time_howard:>12.2f} {time_standard/time_howard:>11.1f}x")

COMPARISON: Standard VFI vs Howard's Policy Improvement

=== VFI with Howard's Policy Improvement (M=0) ===


KeyboardInterrupt: 

In [ ]:
# Visualize convergence comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Convergence by iteration
axes[0].semilogy(vfi_standard.convergence_history, 'b-', label='Standard VFI', alpha=0.7)
axes[0].semilogy(vfi_howard.convergence_history, 'r-', label='VFI + Howard', alpha=0.7)
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('||V - V_old||')
axes[0].set_title('Convergence by Iteration')
axes[0].legend()

# Policy comparison
i_z_mid = model.n_z // 2
policy_standard = vfi_standard.get_policy_k()
policy_howard = vfi_howard.get_policy_k()

axes[1].plot(model.k_grid, policy_standard[:, i_z_mid], 'b-', 
             label='Standard VFI', linewidth=2)
axes[1].plot(model.k_grid, policy_howard[:, i_z_mid], 'r--', 
             label='VFI + Howard', linewidth=2)
axes[1].plot(model.k_grid, model.k_grid, 'k:', alpha=0.3, label='45° line')
axes[1].set_xlabel('Capital (k)')
axes[1].set_ylabel("k'")
axes[1].set_title('Capital Policy Comparison')
axes[1].legend()

plt.tight_layout()
plt.show()

# Check that policies are nearly identical
max_diff = np.max(np.abs(policy_standard - policy_howard))
print(f"\nMax policy difference: {max_diff:.2e}")

## 1.3 Exploiting Monotonicity with Binary Search

**Economic Insight**: The optimal capital policy $k'(k, z)$ is monotonically increasing in $k$. Once we find the optimal $k'$ for capital level $k_i$, we know the optimal $k'$ for $k_{i+1}$ must be at least as large.

**Algorithm Enhancement:**
```
For each z:
  lower_bound = 0
  For k = k_1 to k_N (ascending):
    Search for optimal k' only in [lower_bound, n_k]
    lower_bound = index of optimal k'
```

This reduces the inner loop from $O(n_k)$ to $O(1)$ on average (or $O(\log n_k)$ with binary search).

In [ ]:
# =============================================================================
# 1.4 VFI with Monotonicity Exploitation
# =============================================================================

class VFIMonotone:
    """
    VFI exploiting monotonicity of the policy function.
    """
    
    def __init__(self, model, howard_steps=20):
        self.model = model
        self.howard_steps = howard_steps
        
        self.V = np.zeros((model.n_k, model.n_z))
        self.policy_k_idx = np.zeros((model.n_k, model.n_z), dtype=int)
        self.policy_c = np.zeros((model.n_k, model.n_z))
        
        self._precompute_consumption()
        self.convergence_history = []
        self.timing_info = {}
        
    def _precompute_consumption(self):
        self.consumption_matrix = np.zeros((self.model.n_k, self.model.n_z, self.model.n_k))
        self.feasible = np.zeros((self.model.n_k, self.model.n_z, self.model.n_k), dtype=bool)
        
        for i_k, k in enumerate(self.model.k_grid):
            for i_z, z in enumerate(self.model.z_grid):
                resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                for i_kp, k_prime in enumerate(self.model.k_grid):
                    c = resources - k_prime
                    self.consumption_matrix[i_k, i_z, i_kp] = c
                    self.feasible[i_k, i_z, i_kp] = c > 0
    
    def _howard_step(self):
        V_new = np.zeros_like(self.V)
        for i_k in range(self.model.n_k):
            for i_z in range(self.model.n_z):
                i_kp = self.policy_k_idx[i_k, i_z]
                c = self.consumption_matrix[i_k, i_z, i_kp]
                EV = np.dot(self.model.Pi[i_z, :], self.V[i_kp, :])
                V_new[i_k, i_z] = self.model.utility(c) + self.model.beta * EV
        self.V = V_new
    
    def _bellman_step_monotone(self):
        """
        Bellman step exploiting monotonicity.
        """
        V_new = np.zeros_like(self.V)
        
        for i_z in range(self.model.n_z):
            lower_bound = 0  # Start search from beginning
            
            for i_k in range(self.model.n_k):
                best_val = -np.inf
                best_i_kp = lower_bound
                
                # Search only from lower_bound (monotonicity)
                for i_kp in range(lower_bound, self.model.n_k):
                    if not self.feasible[i_k, i_z, i_kp]:
                        continue
                    
                    c = self.consumption_matrix[i_k, i_z, i_kp]
                    EV = np.dot(self.model.Pi[i_z, :], self.V[i_kp, :])
                    val = self.model.utility(c) + self.model.beta * EV
                    
                    if val > best_val:
                        best_val = val
                        best_i_kp = i_kp
                    elif val < best_val - 1e-10:
                        # Objective is concave, so we can stop
                        break
                
                V_new[i_k, i_z] = best_val
                self.policy_k_idx[i_k, i_z] = best_i_kp
                self.policy_c[i_k, i_z] = self.consumption_matrix[i_k, i_z, best_i_kp]
                
                # Update lower bound for next k (monotonicity)
                lower_bound = best_i_kp
        
        self.V = V_new
    
    def solve(self, tol=1e-6, max_iter=500, verbose=True):
        print(f"\n=== VFI with Monotonicity + Howard (M={self.howard_steps}) ===")
        start_time = time.time()
        
        for iteration in range(max_iter):
            V_old = self.V.copy()
            
            self._bellman_step_monotone()
            
            for _ in range(self.howard_steps):
                self._howard_step()
            
            diff = np.max(np.abs(self.V - V_old))
            self.convergence_history.append(diff)
            
            if verbose and iteration % 10 == 0:
                print(f"Iteration {iteration:4d}: ||V - V_old|| = {diff:.2e}")
            
            if diff < tol:
                elapsed = time.time() - start_time
                self.timing_info['elapsed'] = elapsed
                self.timing_info['iterations'] = iteration + 1
                print(f"\nConverged in {iteration+1} iterations ({elapsed:.2f} seconds)")
                return True
        
        return False
    
    def get_policy_k(self):
        policy_k = np.zeros((self.model.n_k, self.model.n_z))
        for i_k in range(self.model.n_k):
            for i_z in range(self.model.n_z):
                policy_k[i_k, i_z] = self.model.k_grid[self.policy_k_idx[i_k, i_z]]
        return policy_k

In [ ]:
# Compare all three methods
vfi_monotone = VFIMonotone(model, howard_steps=20)
vfi_monotone.solve(tol=1e-6, max_iter=1000, verbose=False)
time_monotone = vfi_monotone.timing_info['elapsed']

print("\n" + "="*60)
print("FINAL COMPARISON")
print("="*60)
print(f"{'Method':<35} {'Time (s)':>12} {'Speedup':>12}")
print("-"*60)
print(f"{'Standard VFI':<35} {time_standard:>12.3f} {'1.0x':>12}")
print(f"{'VFI + Howard':<35} {time_howard:>12.3f} {time_standard/time_howard:>11.1f}x")
print(f"{'VFI + Howard + Monotonicity':<35} {time_monotone:>12.3f} {time_standard/time_monotone:>11.1f}x")

# Part 2: Endogenous Grid Method (EGM)

## 2.1 The Key Insight

**The Problem with Standard Methods**: At each state $(k, z)$, we search for optimal $k'$. This requires evaluating many candidate $k'$ values.

**EGM's Inversion**: Instead of asking "given $k$, what is optimal $k'$?", EGM asks "given $k'$, what $k$ would make this $k'$ optimal?"

**The Algorithm:**
1. Fix a grid of **next-period capital** $k'$ values
2. For each $(k', z)$:
   - Compute expected marginal value: $E[u'(c') \cdot R']$
   - Use Euler equation to find current $c$: $c = (\beta E[\cdot])^{-1/\sigma}$
   - Back out current $k$: $k = (c + k' - (1-\delta)k) / (zk^{\alpha-1})$ → solve for $k$
3. This gives us pairs $(k, c)$ for each $k'$ — an **endogenous** grid!
4. Interpolate to get $c(k)$ on our original grid

**Key Advantage**: No optimization step! The Euler equation is solved analytically.

In [ ]:
# =============================================================================
# 2.2 EGM Implementation
# =============================================================================

class EGMSolver:
    """
    Endogenous Grid Method for the RBC model.
    """
    
    def __init__(self, model):
        self.model = model
        
        # Initialize consumption policy
        self.policy_c = np.zeros((model.n_k, model.n_z))
        for i_k, k in enumerate(model.k_grid):
            for i_z, z in enumerate(model.z_grid):
                resources = z * k**model.alpha + (1 - model.delta) * k
                self.policy_c[i_k, i_z] = 0.5 * resources
        
        # Grid for k' (next period capital) - same as k grid
        self.kp_grid = model.k_grid.copy()
        
        self.convergence_history = []
        self.timing_info = {}
        
    def _egm_step(self):
        """
        One iteration of EGM.
        """
        # Storage for endogenous grid and consumption
        k_endo = np.zeros((self.model.n_k, self.model.n_z))
        c_endo = np.zeros((self.model.n_k, self.model.n_z))
        
        for i_z, z in enumerate(self.model.z_grid):
            for i_kp, kp in enumerate(self.kp_grid):
                
                # Step 1: Compute expected marginal utility * return
                rhs = 0.0
                for i_zp, zp in enumerate(self.model.z_grid):
                    # Next period consumption from current policy
                    cp = np.interp(kp, self.model.k_grid, self.policy_c[:, i_zp])
                    cp = max(cp, 1e-10)
                    
                    # Gross return
                    Rp = self.model.gross_return(kp, zp)
                    
                    # Expected value
                    rhs += self.model.Pi[i_z, i_zp] * self.model.marginal_utility(cp) * Rp
                
                # Step 2: Current consumption from Euler equation
                c = self.model.inv_marginal_utility(self.model.beta * rhs)
                
                # Step 3: Back out current k from budget constraint
                # c + kp = z * k^alpha + (1-delta) * k
                # This is implicit in k, so we solve numerically
                def budget_residual(k):
                    if k <= 0:
                        return np.inf
                    resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                    return resources - c - kp
                
                # Find k that satisfies budget constraint
                try:
                    k_solved = brentq(budget_residual, 1e-6, self.model.k_grid[-1] * 2)
                except:
                    k_solved = self.model.k_grid[i_kp]  # Fallback
                
                k_endo[i_kp, i_z] = k_solved
                c_endo[i_kp, i_z] = c
        
        # Step 4: Interpolate back to original grid
        policy_c_new = np.zeros_like(self.policy_c)
        
        for i_z in range(self.model.n_z):
            # Sort endogenous grid
            sort_idx = np.argsort(k_endo[:, i_z])
            k_sorted = k_endo[sort_idx, i_z]
            c_sorted = c_endo[sort_idx, i_z]
            
            # Remove duplicates and interpolate
            unique_mask = np.concatenate([[True], np.diff(k_sorted) > 1e-10])
            k_unique = k_sorted[unique_mask]
            c_unique = c_sorted[unique_mask]
            
            if len(k_unique) > 1:
                policy_c_new[:, i_z] = np.interp(
                    self.model.k_grid, k_unique, c_unique,
                    left=c_unique[0], right=c_unique[-1]
                )
            else:
                policy_c_new[:, i_z] = self.policy_c[:, i_z]
        
        return policy_c_new
    
    def solve(self, tol=1e-8, max_iter=500, verbose=True):
        """
        Solve using EGM.
        """
        print("\n=== Endogenous Grid Method (EGM) ===")
        start_time = time.time()
        
        for iteration in range(max_iter):
            policy_c_old = self.policy_c.copy()
            
            self.policy_c = self._egm_step()
            
            # Check convergence
            diff = np.max(np.abs(self.policy_c - policy_c_old))
            self.convergence_history.append(diff)
            
            if verbose and iteration % 20 == 0:
                print(f"Iteration {iteration:4d}: ||c - c_old|| = {diff:.2e}")
            
            if diff < tol:
                elapsed = time.time() - start_time
                self.timing_info['elapsed'] = elapsed
                self.timing_info['iterations'] = iteration + 1
                print(f"\nConverged in {iteration+1} iterations ({elapsed:.2f} seconds)")
                return True
        
        print(f"\nWARNING: Did not converge after {max_iter} iterations")
        return False
    
    def get_policy_k(self):
        """Compute capital policy from consumption policy."""
        policy_k = np.zeros_like(self.policy_c)
        for i_k, k in enumerate(self.model.k_grid):
            for i_z, z in enumerate(self.model.z_grid):
                resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                policy_k[i_k, i_z] = resources - self.policy_c[i_k, i_z]
        return policy_k

In [ ]:
# Solve with EGM
egm_solver = EGMSolver(model)
egm_solver.solve(tol=1e-7, max_iter=200, verbose=True)

In [ ]:
# Compare EGM with VFI
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

i_z_mid = model.n_z // 2
egm_policy_k = egm_solver.get_policy_k()
vfi_policy_k = vfi_howard.get_policy_k()

# Consumption policy
axes[0].plot(model.k_grid, vfi_howard.policy_c[:, i_z_mid], 'b-', 
             label='VFI + Howard', linewidth=2)
axes[0].plot(model.k_grid, egm_solver.policy_c[:, i_z_mid], 'r--', 
             label='EGM', linewidth=2)
axes[0].set_xlabel('Capital (k)')
axes[0].set_ylabel('Consumption')
axes[0].set_title('Consumption Policy')
axes[0].legend()

# Capital policy
axes[1].plot(model.k_grid, vfi_policy_k[:, i_z_mid], 'b-', 
             label='VFI + Howard', linewidth=2)
axes[1].plot(model.k_grid, egm_policy_k[:, i_z_mid], 'r--', 
             label='EGM', linewidth=2)
axes[1].plot(model.k_grid, model.k_grid, 'k:', alpha=0.3)
axes[1].set_xlabel('Capital (k)')
axes[1].set_ylabel("k'")
axes[1].set_title('Capital Policy')
axes[1].legend()

# Difference
diff_c = egm_solver.policy_c[:, i_z_mid] - vfi_howard.policy_c[:, i_z_mid]
axes[2].plot(model.k_grid, diff_c * 100, 'g-', linewidth=2)
axes[2].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Capital (k)')
axes[2].set_ylabel('Difference (%)')
axes[2].set_title('Policy Difference (EGM - VFI)')

plt.tight_layout()
plt.show()

# Part 3: Accuracy Assessment with Euler Equation Errors

## 3.1 The Euler Equation Error

The Euler equation must hold exactly for the true policy:

$$u'(c(k,z)) = \beta E\left[ u'(c(k',z')) \cdot R(k',z') \right]$$

For an approximate policy, define the **Euler equation residual**:

$$\varepsilon(k,z) = 1 - \frac{\beta E\left[ u'(c(k',z')) \cdot R(k',z') \right]}{u'(c(k,z))}$$

**Interpretation**: $\varepsilon$ measures the "mistake" in units of consumption. If $\varepsilon = 10^{-3}$, the agent makes a $0.1\%$ consumption error.

**Reporting Convention**: Report $\log_{10}|\varepsilon|$
- $-3$ means $0.1\%$ error
- $-4$ means $0.01\%$ error
- $-6$ means "machine precision"

In [ ]:
# =============================================================================
# 3.2 Euler Equation Error Calculator
# =============================================================================

class EulerErrorCalculator:
    """
    Computes Euler equation errors for accuracy assessment.
    """
    
    def __init__(self, model, policy_c):
        self.model = model
        self.policy_c = policy_c
        
    def compute_errors(self, n_test=1000, seed=42):
        """
        Compute Euler errors at random test points.
        
        Returns array of log10(|euler_error|).
        """
        np.random.seed(seed)
        
        # Random test points
        k_test = np.random.uniform(self.model.k_grid[0], self.model.k_grid[-1], n_test)
        z_idx_test = np.random.randint(0, self.model.n_z, n_test)
        
        errors = np.zeros(n_test)
        
        for i in range(n_test):
            k = k_test[i]
            i_z = z_idx_test[i]
            z = self.model.z_grid[i_z]
            
            # Current consumption (interpolated)
            c = np.interp(k, self.model.k_grid, self.policy_c[:, i_z])
            c = max(c, 1e-10)
            
            # Next period capital
            resources = z * k**self.model.alpha + (1 - self.model.delta) * k
            kp = resources - c
            kp = np.clip(kp, self.model.k_grid[0], self.model.k_grid[-1])
            
            # Expected RHS of Euler equation
            euler_rhs = 0.0
            for i_zp in range(self.model.n_z):
                zp = self.model.z_grid[i_zp]
                cp = np.interp(kp, self.model.k_grid, self.policy_c[:, i_zp])
                cp = max(cp, 1e-10)
                
                Rp = self.model.gross_return(kp, zp)
                euler_rhs += self.model.Pi[i_z, i_zp] * self.model.marginal_utility(cp) * Rp
            
            euler_rhs *= self.model.beta
            
            # Euler error
            euler_lhs = self.model.marginal_utility(c)
            error = 1 - euler_rhs / euler_lhs
            
            errors[i] = np.log10(max(abs(error), 1e-16))
        
        return errors, k_test, z_idx_test
    
    def compute_errors_on_grid(self):
        """
        Compute Euler errors on the full grid.
        """
        errors = np.zeros((self.model.n_k, self.model.n_z))
        
        for i_k, k in enumerate(self.model.k_grid):
            for i_z, z in enumerate(self.model.z_grid):
                c = self.policy_c[i_k, i_z]
                c = max(c, 1e-10)
                
                resources = z * k**self.model.alpha + (1 - self.model.delta) * k
                kp = resources - c
                kp = np.clip(kp, self.model.k_grid[0], self.model.k_grid[-1])
                
                euler_rhs = 0.0
                for i_zp in range(self.model.n_z):
                    zp = self.model.z_grid[i_zp]
                    cp = np.interp(kp, self.model.k_grid, self.policy_c[:, i_zp])
                    cp = max(cp, 1e-10)
                    Rp = self.model.gross_return(kp, zp)
                    euler_rhs += self.model.Pi[i_z, i_zp] * self.model.marginal_utility(cp) * Rp
                
                euler_rhs *= self.model.beta
                euler_lhs = self.model.marginal_utility(c)
                error = 1 - euler_rhs / euler_lhs
                
                errors[i_k, i_z] = np.log10(max(abs(error), 1e-16))
        
        return errors

In [ ]:
# Compute Euler errors for both methods
ee_vfi = EulerErrorCalculator(model, vfi_howard.policy_c)
ee_egm = EulerErrorCalculator(model, egm_solver.policy_c)

errors_vfi, k_test, z_test = ee_vfi.compute_errors(n_test=5000)
errors_egm, _, _ = ee_egm.compute_errors(n_test=5000)

# Summary statistics
print("\n" + "="*60)
print("EULER EQUATION ERRORS (log10 scale)")
print("="*60)
print(f"{'Method':<20} {'Mean':>12} {'Max':>12} {'Min':>12}")
print("-"*60)
print(f"{'VFI + Howard':<20} {np.mean(errors_vfi):>12.2f} {np.max(errors_vfi):>12.2f} {np.min(errors_vfi):>12.2f}")
print(f"{'EGM':<20} {np.mean(errors_egm):>12.2f} {np.max(errors_egm):>12.2f} {np.min(errors_egm):>12.2f}")
print("\nInterpretation: -3 = 0.1% error, -4 = 0.01% error")

In [ ]:
# Visualize Euler errors
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram comparison
axes[0, 0].hist(errors_vfi, bins=50, alpha=0.7, label='VFI + Howard', density=True)
axes[0, 0].hist(errors_egm, bins=50, alpha=0.7, label='EGM', density=True)
axes[0, 0].axvline(-3, color='red', linestyle='--', label='0.1% error')
axes[0, 0].axvline(-4, color='green', linestyle='--', label='0.01% error')
axes[0, 0].set_xlabel('log₁₀|Euler Error|')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Distribution of Euler Errors')
axes[0, 0].legend()

# Errors on grid (VFI)
errors_grid_vfi = ee_vfi.compute_errors_on_grid()
im1 = axes[0, 1].imshow(errors_grid_vfi.T, aspect='auto', origin='lower',
                         extent=[model.k_grid[0], model.k_grid[-1], 0, model.n_z-1],
                         cmap='RdYlGn_r', vmin=-6, vmax=-2)
axes[0, 1].set_xlabel('Capital (k)')
axes[0, 1].set_ylabel('Productivity State (z index)')
axes[0, 1].set_title('Euler Errors: VFI + Howard')
plt.colorbar(im1, ax=axes[0, 1], label='log₁₀|error|')

# Errors on grid (EGM)
errors_grid_egm = ee_egm.compute_errors_on_grid()
im2 = axes[1, 0].imshow(errors_grid_egm.T, aspect='auto', origin='lower',
                         extent=[model.k_grid[0], model.k_grid[-1], 0, model.n_z-1],
                         cmap='RdYlGn_r', vmin=-6, vmax=-2)
axes[1, 0].set_xlabel('Capital (k)')
axes[1, 0].set_ylabel('Productivity State (z index)')
axes[1, 0].set_title('Euler Errors: EGM')
plt.colorbar(im2, ax=axes[1, 0], label='log₁₀|error|')

# Error by capital level
axes[1, 1].plot(model.k_grid, errors_grid_vfi[:, model.n_z//2], 'b-', 
                label='VFI + Howard', linewidth=2)
axes[1, 1].plot(model.k_grid, errors_grid_egm[:, model.n_z//2], 'r--', 
                label='EGM', linewidth=2)
axes[1, 1].axhline(-3, color='gray', linestyle=':', alpha=0.5)
axes[1, 1].axhline(-4, color='gray', linestyle=':', alpha=0.5)
axes[1, 1].set_xlabel('Capital (k)')
axes[1, 1].set_ylabel('log₁₀|Euler Error|')
axes[1, 1].set_title('Euler Errors at Median Productivity')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# Part 4: Sensitivity Analysis

## 4.1 How Do Numerical Choices Affect Results?

Key questions:
1. **Grid density**: How many points do we need?
2. **Tolerance**: How tight should convergence be?
3. **Interpolation**: Linear vs. cubic vs. higher-order?

In [ ]:
# =============================================================================
# 4.2 Grid Density Analysis
# =============================================================================

grid_sizes = [25, 50, 100, 200, 400]
results_grid = []

print("\nGrid Density Analysis")
print("="*60)

for n_k in grid_sizes:
    # Create model with different grid size
    model_test = RBCModel(n_k=n_k, n_z=7)
    
    # Solve with VFI + Howard
    vfi_test = VFIHoward(model_test, howard_steps=20)
    vfi_test.solve(tol=1e-7, max_iter=500, verbose=False)
    
    # Compute Euler errors
    ee_test = EulerErrorCalculator(model_test, vfi_test.policy_c)
    errors, _, _ = ee_test.compute_errors(n_test=2000)
    
    results_grid.append({
        'n_k': n_k,
        'time': vfi_test.timing_info['elapsed'],
        'mean_error': np.mean(errors),
        'max_error': np.max(errors)
    })
    
    print(f"n_k={n_k:4d}: time={vfi_test.timing_info['elapsed']:.2f}s, "
          f"mean error={np.mean(errors):.2f}, max error={np.max(errors):.2f}")

In [ ]:
# Visualize grid density tradeoffs
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

grid_sizes_arr = np.array([r['n_k'] for r in results_grid])
times = np.array([r['time'] for r in results_grid])
mean_errors = np.array([r['mean_error'] for r in results_grid])
max_errors = np.array([r['max_error'] for r in results_grid])

# Time vs Grid Size
axes[0].loglog(grid_sizes_arr, times, 'bo-', markersize=10, linewidth=2)
axes[0].set_xlabel('Number of Grid Points')
axes[0].set_ylabel('Solution Time (seconds)')
axes[0].set_title('Computational Cost vs Grid Density')
axes[0].grid(True, alpha=0.3)

# Accuracy vs Grid Size
axes[1].semilogx(grid_sizes_arr, mean_errors, 'b-o', markersize=10, linewidth=2, label='Mean Error')
axes[1].semilogx(grid_sizes_arr, max_errors, 'r-s', markersize=10, linewidth=2, label='Max Error')
axes[1].axhline(-3, color='gray', linestyle='--', alpha=0.5, label='0.1% threshold')
axes[1].axhline(-4, color='gray', linestyle=':', alpha=0.5, label='0.01% threshold')
axes[1].set_xlabel('Number of Grid Points')
axes[1].set_ylabel('log₁₀|Euler Error|')
axes[1].set_title('Accuracy vs Grid Density')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Part 5: Impulse Response Analysis

## 5.1 Economic Plausibility Check

A good numerical solution should produce economically sensible impulse responses:
- Positive productivity shock → Output, consumption, investment all rise
- Consumption smoother than output
- Investment more volatile than output
- Gradual return to steady state

In [ ]:
# =============================================================================
# 5.2 Impulse Response Function Generator
# =============================================================================

def compute_irf(model, policy_c, shock_size=1, T=40):
    """
    Compute impulse response to a one-standard-deviation productivity shock.
    
    shock_size: number of grid points to move from median
    """
    # Baseline path (no shock)
    k_base = np.zeros(T+1)
    y_base = np.zeros(T)
    c_base = np.zeros(T)
    i_base = np.zeros(T)
    
    k_base[0] = model.k_ss
    z_idx_base = model.n_z // 2  # Median productivity
    
    for t in range(T):
        k = k_base[t]
        z = model.z_grid[z_idx_base]
        
        c = np.interp(k, model.k_grid, policy_c[:, z_idx_base])
        y = z * k ** model.alpha
        resources = y + (1 - model.delta) * k
        kp = resources - c
        
        y_base[t] = y
        c_base[t] = c
        i_base[t] = kp - (1 - model.delta) * k
        k_base[t+1] = kp
    
    # Shocked path
    k_shock = np.zeros(T+1)
    y_shock = np.zeros(T)
    c_shock = np.zeros(T)
    i_shock = np.zeros(T)
    z_idx_shock = np.zeros(T, dtype=int)
    
    k_shock[0] = model.k_ss
    z_idx_shock[0] = min(model.n_z // 2 + shock_size, model.n_z - 1)  # Positive shock
    
    for t in range(T):
        k = k_shock[t]
        z_idx = z_idx_shock[t]
        z = model.z_grid[z_idx]
        
        c = np.interp(k, model.k_grid, policy_c[:, z_idx])
        y = z * k ** model.alpha
        resources = y + (1 - model.delta) * k
        kp = resources - c
        
        y_shock[t] = y
        c_shock[t] = c
        i_shock[t] = kp - (1 - model.delta) * k
        k_shock[t+1] = kp
        
        # Productivity reverts to mean (deterministically for clean IRF)
        if t < T - 1:
            # Expected next z index (mean reversion)
            expected_log_z = model.rho * model.log_z_grid[z_idx]
            z_idx_shock[t+1] = np.argmin(np.abs(model.log_z_grid - expected_log_z))
    
    # Compute percentage deviations from baseline
    irf = {
        'Y': 100 * (y_shock / y_base - 1),
        'C': 100 * (c_shock / c_base - 1),
        'I': 100 * (i_shock / i_base - 1),
        'K': 100 * (k_shock[:-1] / k_base[:-1] - 1),
        'z': model.z_grid[z_idx_shock]
    }
    
    return irf

In [ ]:
# Compute and plot IRFs
irf_vfi = compute_irf(model, vfi_howard.policy_c, shock_size=2, T=40)
irf_egm = compute_irf(model, egm_solver.policy_c, shock_size=2, T=40)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

T = len(irf_vfi['Y'])
periods = np.arange(T)

# Output
axes[0, 0].plot(periods, irf_vfi['Y'], 'b-', label='VFI + Howard', linewidth=2)
axes[0, 0].plot(periods, irf_egm['Y'], 'r--', label='EGM', linewidth=2)
axes[0, 0].axhline(0, color='gray', linestyle=':', alpha=0.5)
axes[0, 0].set_xlabel('Periods after shock')
axes[0, 0].set_ylabel('% deviation from baseline')
axes[0, 0].set_title('Output Response')
axes[0, 0].legend()

# Consumption
axes[0, 1].plot(periods, irf_vfi['C'], 'b-', label='VFI + Howard', linewidth=2)
axes[0, 1].plot(periods, irf_egm['C'], 'r--', label='EGM', linewidth=2)
axes[0, 1].axhline(0, color='gray', linestyle=':', alpha=0.5)
axes[0, 1].set_xlabel('Periods after shock')
axes[0, 1].set_ylabel('% deviation from baseline')
axes[0, 1].set_title('Consumption Response')
axes[0, 1].legend()

# Investment
axes[1, 0].plot(periods, irf_vfi['I'], 'b-', label='VFI + Howard', linewidth=2)
axes[1, 0].plot(periods, irf_egm['I'], 'r--', label='EGM', linewidth=2)
axes[1, 0].axhline(0, color='gray', linestyle=':', alpha=0.5)
axes[1, 0].set_xlabel('Periods after shock')
axes[1, 0].set_ylabel('% deviation from baseline')
axes[1, 0].set_title('Investment Response')
axes[1, 0].legend()

# Capital
axes[1, 1].plot(periods, irf_vfi['K'], 'b-', label='VFI + Howard', linewidth=2)
axes[1, 1].plot(periods, irf_egm['K'], 'r--', label='EGM', linewidth=2)
axes[1, 1].axhline(0, color='gray', linestyle=':', alpha=0.5)
axes[1, 1].set_xlabel('Periods after shock')
axes[1, 1].set_ylabel('% deviation from baseline')
axes[1, 1].set_title('Capital Response')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\nEconomic Plausibility Checks:")
print(f"✓ Output rises on impact: {irf_vfi['Y'][0]:.2f}%")
print(f"✓ Consumption smoother: max C response = {max(irf_vfi['C']):.2f}% < max Y = {max(irf_vfi['Y']):.2f}%")
print(f"✓ Investment more volatile: max I response = {max(irf_vfi['I']):.2f}%")
print(f"✓ Gradual convergence: Y at t=20 = {irf_vfi['Y'][20]:.2f}%")

# Summary and Key Takeaways

## What We Learned

### Part 1: Accelerating VFI
- **Howard's Policy Improvement**: Dramatically reduces computation time (10-100x speedup)
- **Monotonicity**: Further gains by exploiting economic structure
- **Key insight**: The policy converges faster than the value function

### Part 2: Endogenous Grid Method
- **Eliminates optimization**: Transforms the problem from finding optimal $k'$ to inverting the Euler equation
- **Trade-off**: Requires solving for endogenous grid points
- **Best for**: Models with well-behaved Euler equations

### Part 3: Accuracy Assessment
- **Euler equation errors**: The gold standard for accuracy measurement
- **Interpretation**: $\log_{10}$ scale makes errors comparable
- **Target**: Mean errors below $-3$ (0.1% consumption error) for most applications

### Part 4: Sensitivity Analysis
- **Grid density**: Diminishing returns beyond ~100-200 points for simple models
- **Trade-off**: Computational cost grows faster than accuracy improves

### Part 5: Impulse Responses
- **Economic validation**: Does the solution make economic sense?
- **Key patterns**: Consumption smoothing, investment volatility, mean reversion

---

## Looking Ahead (Lab 9)

Next lab: **Perturbation and Projection Methods**
- Linear and higher-order perturbation around steady state
- Projection methods (Galerkin, collocation)
- When to use which method

---

## 🎯 Research Architect Reflection

Today we demonstrated the full validation pipeline:

1. **Speed**: Does the algorithm run fast enough? (Timing comparisons)
2. **Accuracy**: Are the results numerically precise? (Euler equation errors)
3. **Robustness**: Do results depend on numerical choices? (Sensitivity analysis)
4. **Economic Sense**: Are the results plausible? (Impulse responses)

A Research Architect's value lies in knowing **which questions to ask** and **how to interpret answers**. The implementation is increasingly automatable; the judgment is not.